# Χρησιμοποιήθηκαν 2 free pass

In [1]:
import pandas as pd
import numpy as np

import gensim 
import gensim.models
from gensim.models import Word2Vec
from gensim import utils

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegressionCV
from sklearn.utils import shuffle
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import precision_score

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jimmy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
revDf = pd.read_csv('reviewsRest.csv')
restF = pd.read_csv('philly_restaurants_categories.csv')

In [3]:
restF.sort_values(by=['business_id'], inplace=True)
restF.head

<bound method NDFrame.head of                 business_id  category
219  -AbzJTLQdbdQrhRzQLgsKA  Japanese
400  -HxLFWKVgXSU8JlR21PBkg   Italian
23   -LmhsdQproqCf5EQoD06rQ  Japanese
154  -MkngKKkTIVfnUbq2S1ucQ   Italian
233  -PMXnNJ1D67NkAupRHNkpQ   Italian
..                      ...       ...
39   zUJMvjK6aBeQtVCowZ85-w  Japanese
240  zeounyPVXFZEz1c9KtptLA   Italian
804  zgX8sYCRGVJ9M5LETpJ60A   Burgers
401  zqisPpgCURrgLf4TVnI8RQ  Japanese
55   zzyx5x0Z7xXWWvWnZFuxlQ   Italian

[951 rows x 2 columns]>

<span style="color:black; font-weight:700;font-size:18px"> Αυτήν την στιγμή έχουμε ένα DataFrame με κάθε business_id και το review του. Κάνουμε groupby με τα business_id, κρατάμε σε μια λίστα τα business_ids και δημιουργούμε μια νέα στήλη στην οποία έχουμε κάνει join κάθε review της επιχείρησης. </span>

In [4]:
bus_groups = revDf.groupby('business_id')
business_names = []
text_joined = []

for business_id, bus_data in bus_groups:
    #print(business_id)
    business_names.append(business_id)
    #print(bus_data['text'])
    text_joined.append(' '.join(bus_data["text"]))
    

<span style="color:black; font-weight:700;font-size:18px"> Δημιουργούμε ένα DataFrame bus_reviews με δύο στήλες, στην πρώτη το business_id και στην άλλη όλα τα review ενωμένα. </span>

In [5]:
bus_reviews = pd.DataFrame([])
bus_reviews['business_id'] = pd.Series(business_names)
bus_reviews['text'] = pd.Series(text_joined)
bus_reviews.head

<bound method NDFrame.head of                 business_id                                               text
0    -AbzJTLQdbdQrhRzQLgsKA  Kingyo has summer appetizer specials for $3.00...
1    -HxLFWKVgXSU8JlR21PBkg  "Carluccio's: Way Delish"\n\nEver find yoursel...
2    -LmhsdQproqCf5EQoD06rQ  Down in south Philly for a meeting at my daugh...
3    -MkngKKkTIVfnUbq2S1ucQ  Dee-frickin'-licious thick crust pizza!!! I or...
4    -PMXnNJ1D67NkAupRHNkpQ  I've always preferred domino's over pizza hut!...
..                      ...                                                ...
946  zUJMvjK6aBeQtVCowZ85-w  Excelent! Food is wondeful, hot, and fresh! Th...
947  zeounyPVXFZEz1c9KtptLA  Pizza, wings, steaks are good.. chicken finger...
948  zgX8sYCRGVJ9M5LETpJ60A  I popped in there one afternoon to pick up som...
949  zqisPpgCURrgLf4TVnI8RQ  This review is for poke bowl. Very fresh salad...
950  zzyx5x0Z7xXWWvWnZFuxlQ  Maybe the pizza is good here... but I can real...

[951 rows x 2 columns

<span style="color:black; font-weight:700;font-size:18px"> Τώρα για να αναθέσουμε το category σε κάθε business_id ταξινομούμε το restF DataFrame και θέτουμε στο bus_review ως νέα στήλη την στήλη category_label από το restF. </span>

In [6]:
#restF.sort_values(by=['business_id'], inplace=True)
bus_reviews['category'] = restF['category'].tolist()
bus_reviews.head

<bound method NDFrame.head of                 business_id  \
0    -AbzJTLQdbdQrhRzQLgsKA   
1    -HxLFWKVgXSU8JlR21PBkg   
2    -LmhsdQproqCf5EQoD06rQ   
3    -MkngKKkTIVfnUbq2S1ucQ   
4    -PMXnNJ1D67NkAupRHNkpQ   
..                      ...   
946  zUJMvjK6aBeQtVCowZ85-w   
947  zeounyPVXFZEz1c9KtptLA   
948  zgX8sYCRGVJ9M5LETpJ60A   
949  zqisPpgCURrgLf4TVnI8RQ   
950  zzyx5x0Z7xXWWvWnZFuxlQ   

                                                  text  category  
0    Kingyo has summer appetizer specials for $3.00...  Japanese  
1    "Carluccio's: Way Delish"\n\nEver find yoursel...   Italian  
2    Down in south Philly for a meeting at my daugh...  Japanese  
3    Dee-frickin'-licious thick crust pizza!!! I or...   Italian  
4    I've always preferred domino's over pizza hut!...   Italian  
..                                                 ...       ...  
946  Excelent! Food is wondeful, hot, and fresh! Th...  Japanese  
947  Pizza, wings, steaks are good.. chicken finger...   Ital

<span style="color:black; font-weight:700;font-size:18px"> Τώρα με την χρήση lambda function και την λίστα από τα stopwords που κατεβάσαμε στην αρχή, αν μια λέξη δεν περιέχεται στο stopwords list, την κρατάμε. Το νέο κείμενο που δημιουργείται, κρατιέται σε μια νέα στήλη. </span>

In [7]:
stop = stopwords.words('english')
bus_reviews['text_without_sw'] = bus_reviews['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
bus_reviews.head

<bound method NDFrame.head of                 business_id  \
0    -AbzJTLQdbdQrhRzQLgsKA   
1    -HxLFWKVgXSU8JlR21PBkg   
2    -LmhsdQproqCf5EQoD06rQ   
3    -MkngKKkTIVfnUbq2S1ucQ   
4    -PMXnNJ1D67NkAupRHNkpQ   
..                      ...   
946  zUJMvjK6aBeQtVCowZ85-w   
947  zeounyPVXFZEz1c9KtptLA   
948  zgX8sYCRGVJ9M5LETpJ60A   
949  zqisPpgCURrgLf4TVnI8RQ   
950  zzyx5x0Z7xXWWvWnZFuxlQ   

                                                  text  category  \
0    Kingyo has summer appetizer specials for $3.00...  Japanese   
1    "Carluccio's: Way Delish"\n\nEver find yoursel...   Italian   
2    Down in south Philly for a meeting at my daugh...  Japanese   
3    Dee-frickin'-licious thick crust pizza!!! I or...   Italian   
4    I've always preferred domino's over pizza hut!...   Italian   
..                                                 ...       ...   
946  Excelent! Food is wondeful, hot, and fresh! Th...  Japanese   
947  Pizza, wings, steaks are good.. chicken finger..

In [8]:
bus_reviews = shuffle(bus_reviews)
folds = KFold(n_splits=5)
train_folds = []
test_folds = []
for trainIndex, testIndex in folds.split(bus_reviews):
    # print(trainIndex, testIndex)
    xTrain = bus_reviews.loc[trainIndex, :]
    xTest = bus_reviews.loc[testIndex, :]
    train_folds.append(xTrain)
    test_folds.append(xTest)
    print(xTrain.shape, xTest.shape)

(760, 4) (191, 4)
(761, 4) (190, 4)
(761, 4) (190, 4)
(761, 4) (190, 4)
(761, 4) (190, 4)


<span style="color:black; font-weight:700;font-size:18px"> Τώρα για κάθε business_id έχουμε την ένωση όλων των reviews χωρίς stopwords, και για αυτή την ένωση κειμένων βγάζουμε σε ένα array τα αποτελέσματα του tfid αλγορίθμου. Μετά, κρατάμε στο feature_names τις λέξεις του κειμένου που θα χρησιμοποιηθούν αργότερα. </span>

In [9]:
############################# VERSION 2 #######################################
train_set = []
test_set = []
for i in range(len(train_folds)):
    v = TfidfVectorizer(stop_words = 'english', min_df = 0.2, max_df = 0.8, max_features=3000)
    #print("aa")
    x = v.fit_transform(train_folds[i]['text_without_sw']).toarray()
    train_set.append(x)
    y = v.transform(test_folds[i]['text_without_sw']).toarray()
    test_set.append(y)
#x = v.fit_transform(bus_reviews['text_without_sw']).toarray()

feature_names = v.get_feature_names()
############################# VERSION 2 #######################################

In [10]:
print(len(train_set[0][0]),len(test_set[0][0]))
print(len(train_set[1][0]),len(test_set[1][0]))
print(len(train_set[2][0]),len(test_set[2][0]))
print(len(train_set[3][0]),len(test_set[3][0]))

2011 2011
1989 1989
2084 2084
2012 2012


In [11]:
# oi grammes einai oi pragmatikes times
# 1h grammh: 316 ta burgers
# 2h grammh: 431 ta italika
# 3h grammh: 204 ta iapwnika

<span style="color:black; font-weight:700;font-size:18px"> Logistic </span>

In [12]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = LogisticRegression(random_state=16)
    clf.fit(train_set[i],train_folds[i]['category'])
    y_pred = clf.predict(test_set[i])
    acc.append(metrics.accuracy_score(y_pred, test_folds[i]['category']))
    prec.append(metrics.precision_score(y_pred, test_folds[i]['category'], average=None))
    rec.append(metrics.recall_score(y_pred, test_folds[i]['category'], average=None))
    f1.append(metrics.f1_score(y_pred, test_folds[i]['category'], average=None))
    conf_mat = confusion_matrix(y_pred, test_folds[i]['category'])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

[[56  5  2]
 [ 7 80  0]
 [ 0  0 38]]
Accuracy:   0.9179608707632957
Recall:    [0.88651472 0.91335592 0.983467  ]
Precision: [0.88410181 0.93380337 0.93068206]
F1_Score:  [0.88487407 0.92310786 0.95603393]


<span style="color:black; font-weight:700;font-size:18px"> SVM </span>

In [13]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = svm.SVC(kernel='linear', C=1, random_state=42)
    clf.fit(train_set[i],train_folds[i]['category'])
    y_pred = clf.predict(test_set[i])
    acc.append(metrics.accuracy_score(y_pred, test_folds[i]['category']))
    prec.append(metrics.precision_score(y_pred, test_folds[i]['category'], average=None))
    rec.append(metrics.recall_score(y_pred, test_folds[i]['category'], average=None))
    f1.append(metrics.f1_score(y_pred, test_folds[i]['category'], average=None))
    conf_mat = confusion_matrix(y_pred, test_folds[i]['category'])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

[[55  5  1]
 [ 6 80  1]
 [ 0  0 38]]
Accuracy:   0.9137558556076053
Recall:    [0.88198532 0.91224145 0.96923228]
Precision: [0.87822351 0.92914406 0.93034946]
F1_Score:  [0.87998192 0.9202957  0.94866767]


<span style="color:black; font-weight:700;font-size:18px"> k-NN </span>

In [14]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = KNeighborsClassifier(n_neighbors = 20)
    clf.fit(train_set[i],train_folds[i]['category'])
    y_pred = clf.predict(test_set[i])
    acc.append(metrics.accuracy_score(y_pred, test_folds[i]['category']))
    prec.append(metrics.precision_score(y_pred, test_folds[i]['category'], average=None))
    rec.append(metrics.recall_score(y_pred, test_folds[i]['category'], average=None))
    f1.append(metrics.f1_score(y_pred, test_folds[i]['category'], average=None))
    conf_mat = confusion_matrix(y_pred, test_folds[i]['category'])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

[[52  6  1]
 [ 9 79  1]
 [ 0  0 38]]
Accuracy:   0.8979774042435933
Recall:    [0.87977727 0.88109859 0.96895527]
Precision: [0.83247528 0.92208556 0.94136582]
F1_Score:  [0.85462505 0.90043428 0.95436458]


D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

<span style="color:black; font-weight:700;font-size:18px"> Χρησιμοποιούμε την έτοιμη συνάρτηση για k-fold logistic και κρατάμε τα βάρη σε ένα DataFrame, όπου κάθε γραμμή αντιστοιχεί και σε κάθε label (Burger, Italian, Japanese) </span>

In [15]:
%%capture
logistic = LogisticRegression(random_state=16)
logistic.fit(train_set[i],train_folds[i]['category'])
w = logistic.coef_ #weights of each feature

df = pd.DataFrame(w, columns=feature_names)

# Change the row indexes
df.index = ['0', '1', '2']

<span style="color:black; font-weight:700;font-size:18px"> Οι 10 πιο σημαντικές και οι πιο ξένες λέξεις για τα εστιατόρια Burgers </span>

In [16]:
rslt_df = df.sort_values(by = '0', axis = 1, ascending = False)
rslt_ll = rslt_df.iloc[0]
print("MOST valuable Burgers restaurants words:")
print(rslt_ll[:21])
print("########################################")
rslt_df = df.sort_values(by = '0', axis = 1, ascending = True)
rslt_ll = rslt_df.iloc[0]
print("LEAST valuable Burgers restaurants words:")
print(rslt_ll[:21])

MOST valuable Burgers restaurants words:
fries          2.134281
burger         2.103392
burgers        1.229712
drive          1.168020
breakfast      1.092934
wings          0.916595
mac            0.832220
cheesesteak    0.694895
sandwich       0.670733
window         0.569602
dirty          0.524541
bacon          0.516179
dog            0.462561
worst          0.455936
horrible       0.451668
waited         0.444631
buffalo        0.437612
beer           0.416832
location       0.401911
employees      0.400823
late           0.390432
Name: 0, dtype: float64
########################################
LEAST valuable Burgers restaurants words:
sushi       -1.439078
italian     -1.308981
pasta       -1.156885
pork        -1.037388
rice        -0.892452
wine        -0.806721
japanese    -0.728176
pizza       -0.723810
spicy       -0.591858
meatballs   -0.585062
dinner      -0.566197
noodles     -0.559680
broth       -0.558542
dishes      -0.541073
calamari    -0.531610
bowl        -0.514

<span style="color:black; font-weight:700;font-size:18px"> Οι 10 πιο σημαντικές και οι πιο ξένες λέξεις για τα εστιατόρια Italian </span>

In [17]:
rslt_df = df.sort_values(by = '1', axis = 1, ascending = False)
rslt_ll = rslt_df.iloc[0]
print("MOST valuable Italian restaurants words:")
print(rslt_ll[:21])
print("########################################")
rslt_df = df.sort_values(by = '1', axis = 1, ascending = True)
rslt_ll = rslt_df.iloc[0]
print("LEAST valuable Italian restaurants words:")
print(rslt_ll[:21])

MOST valuable Italian restaurants words:
pizza        -0.723810
italian      -1.308981
pasta        -1.156885
wine         -0.806721
bread        -0.428513
meatballs    -0.585062
dinner       -0.566197
hoagie       -0.101680
coffee       -0.264474
gnocchi      -0.390034
salad        -0.455406
byob         -0.476074
club         -0.463857
parm         -0.373843
calamari     -0.531610
veal         -0.323491
mozzarella   -0.391329
meatball     -0.338559
sandwiches    0.087374
ravioli      -0.253432
crust        -0.255564
Name: 0, dtype: float64
########################################
LEAST valuable Italian restaurants words:
sushi       -1.439078
fries        2.134281
burger       2.103392
burgers      1.229712
drive        1.168020
noodles     -0.559680
broth       -0.558542
chinese     -0.408421
bowl        -0.514072
japanese    -0.728176
ice          0.147607
rice        -0.892452
bbq          0.070365
wings        0.916595
mac          0.832220
dumplings   -0.511178
salmon       0.10

<span style="color:black; font-weight:700;font-size:18px"> Οι 10 πιο σημαντικές και οι πιο ξένες λέξεις για τα εστιατόρια Japanese </span>

In [18]:
rslt_df = df.sort_values(by = '2', axis = 1, ascending = False)
rslt_ll = rslt_df.iloc[0]
print("MOST valuable Japanese restaurants words:")
print(rslt_ll[:21])
print("########################################")
rslt_df = df.sort_values(by = '2', axis = 1, ascending = True)
rslt_ll = rslt_df.iloc[0]
print("LEAST valuable Japanese restaurants words:")
print(rslt_ll[:21])

MOST valuable Japanese restaurants words:
sushi       -1.439078
rice        -0.892452
japanese    -0.728176
noodles     -0.559680
broth       -0.558542
bowl        -0.514072
chinese     -0.408421
pork        -1.037388
dumplings   -0.511178
spicy       -0.591858
tea         -0.436330
soup        -0.508118
rolls       -0.445268
roll        -0.366842
fried       -0.255254
beef        -0.222559
tuna        -0.308703
bowls       -0.271210
room        -0.440289
noodle      -0.198795
shrimp      -0.012936
Name: 0, dtype: float64
########################################
LEAST valuable Japanese restaurants words:
pizza         -0.723810
burger         2.103392
sandwich       0.670733
italian       -1.308981
breakfast      1.092934
fries          2.134281
pasta         -1.156885
bread         -0.428513
hoagie        -0.101680
sandwiches     0.087374
burgers        1.229712
steak          0.211505
wings          0.916595
cheesesteak    0.694895
brunch         0.194898
drive          1.168020
coff

# CBOW

In [19]:
# TRAIN PARAMS
# i: 0->4
# train_folds[i],train_folds[i]['category']

In [20]:
X_train_gsim_L = []
y_train_gsim_L = []
for i in range(5):
    train_gsim = [utils.simple_preprocess(x) for x in train_folds[i]['text']]
    train_data_labels = [(x,y) for (x,y) in zip(train_gsim, train_folds[i]['category']) if len(x) > 0]
    X_train_gsim = [x for (x,y) in train_data_labels]
    y_train_gsim = [y for (x,y) in train_data_labels]
    X_train_gsim_L.append(X_train_gsim)
    y_train_gsim_L.append(y_train_gsim)

In [21]:
# TEST PARAMS
# i: 0->4
# test_folds[i],test_folds[i]['category']

In [22]:
X_test_gsim_L = []
y_test_gsim_L = []
for i in range(5):
    test_gsim = [utils.simple_preprocess(x) for x in test_folds[i]['text']]
    test_data_labels = [(x,y) for (x,y) in zip(test_gsim, test_folds[i]['category']) if len(x) > 0]
    X_test_gsim = [x for (x,y) in test_data_labels]
    y_test_gsim = [y for (x,y) in test_data_labels]
    X_test_gsim_L.append(X_test_gsim)
    y_test_gsim_L.append(y_test_gsim)

In [23]:
print(len(X_train_gsim_L[0]),len(y_train_gsim_L[0]))
print(len(X_test_gsim_L),len(y_test_gsim_L))

760 760
5 5


In [24]:
embedding_size = 50
cbow_list = []
# sg=0: cbow, sg=1: skipgram
for i in range(5):
    cbow_model = gensim.models.Word2Vec(X_train_gsim_L[i], min_count = 1, window = 10, sg=0)
    cbow_list.append(cbow_model)

In [25]:
X_train_cbow = []
for i in range(5):
    X_train_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y]).mean(axis = 0) for y in X_train_gsim_L[i]]
    X_train_cbow.append(X_train_cbow_temp)

In [26]:
X_test_cbow = []
for i in range(5):
    X_test_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y if x in cbow_list[i].wv]).mean(axis = 0) for y in X_test_gsim_L[i]]
    X_test_cbow.append(X_test_cbow_temp)

<span style="color:black; font-weight:700;font-size:18px"> Logistic </span>

In [27]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = LogisticRegression(random_state=16,solver='lbfgs', max_iter=200)
    clf.fit(X_train_cbow[i],y_train_gsim_L[i])
    y_pred = clf.predict(X_test_cbow[i])
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

[[55  6  0]
 [ 7 79  1]
 [ 0  0 39]]
Accuracy:   0.9137558556076053
Recall:    [0.88720992 0.90247666 0.98430324]
Precision: [0.87451297 0.91461565 0.96567362]
F1_Score:  [0.87995274 0.90815591 0.97471536]


<span style="color:black; font-weight:700;font-size:18px"> SVM </span>

In [28]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = svm.SVC(kernel='linear', C=1, random_state=42)
    clf.fit(X_train_cbow[i],y_train_gsim_L[i])
    y_pred = clf.predict(X_test_cbow[i])
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

[[56  7  0]
 [ 6 78  0]
 [ 0  0 39]]
Accuracy:   0.9158611187655001
Recall:    [0.88067963 0.91500587 0.97892603]
Precision: [0.88750066 0.90979389 0.96534103]
F1_Score:  [0.88309351 0.91178369 0.97193004]


<span style="color:black; font-weight:700;font-size:18px"> k-NN </span>

In [29]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = KNeighborsClassifier(n_neighbors = 20, metric='cosine')
    clf.fit(X_train_cbow[i],y_train_gsim_L[i])
    y_pred = clf.predict(X_test_cbow[i])
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

[[52  8  1]
 [10 77  1]
 [ 0  0 38]]
Accuracy:   0.8801047120418849
Recall:    [0.83774083 0.87062739 0.9688764 ]
Precision: [0.82747898 0.89260321 0.93024239]
F1_Score:  [0.83204988 0.88144216 0.94845997]


D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

# Skipgram

In [30]:
########## MAYBE REPETITIVE CODE, WE HAD IT ALREADY IN CBOW
# TRAIN PARAMS
# i: 0->4
# train_folds[i],train_folds[i]['category']

X_train_gsim_L = []
y_train_gsim_L = []
for i in range(5):
    train_gsim = [utils.simple_preprocess(x) for x in train_folds[i]['text']]
    train_data_labels = [(x,y) for (x,y) in zip(train_gsim, train_folds[i]['category']) if len(x) > 0]
    X_train_gsim = [x for (x,y) in train_data_labels]
    y_train_gsim = [y for (x,y) in train_data_labels]
    X_train_gsim_L.append(X_train_gsim)
    y_train_gsim_L.append(y_train_gsim)

# TEST PARAMS
# i: 0->4
# test_folds[i],test_folds[i]['category']

X_test_gsim_L = []
y_test_gsim_L = []
for i in range(5):
    test_gsim = [utils.simple_preprocess(x) for x in test_folds[i]['text']]
    test_data_labels = [(x,y) for (x,y) in zip(test_gsim, test_folds[i]['category']) if len(x) > 0]
    X_test_gsim = [x for (x,y) in test_data_labels]
    y_test_gsim = [y for (x,y) in test_data_labels]
    X_test_gsim_L.append(X_test_gsim)
    y_test_gsim_L.append(y_test_gsim)

In [31]:
embedding_size = 50
cbow_list = []
# sg=0: cbow, sg=1: skipgram
for i in range(5):
    print("Fold")
    cbow_model = gensim.models.Word2Vec(X_train_gsim_L[i], min_count = 1, window = 10, sg=1)
    cbow_list.append(cbow_model)

Fold
Fold
Fold
Fold
Fold


In [32]:
X_train_cbow = []
for i in range(5):
    X_train_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y]).mean(axis = 0) for y in X_train_gsim_L[i]]
    X_train_cbow.append(X_train_cbow_temp)

In [33]:
X_test_cbow = []
for i in range(5):
    X_test_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y if x in cbow_list[i].wv]).mean(axis = 0) for y in X_test_gsim_L[i]]
    X_test_cbow.append(X_test_cbow_temp)

<span style="color:black; font-weight:700;font-size:18px"> Logistic </span>

In [34]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = LogisticRegression(random_state=16,solver='lbfgs', max_iter=200)
    clf.fit(X_train_cbow[i],y_train_gsim_L[i])
    y_pred = clf.predict(X_test_cbow[i])
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

[[48  6  1]
 [15 79  4]
 [ 0  0 35]]
Accuracy:   0.8569633507853404
Recall:    [0.86147268 0.80708035 0.98267974]
Precision: [0.75764955 0.91859939 0.87784126]
F1_Score:  [0.80598849 0.85873799 0.92612787]


<span style="color:black; font-weight:700;font-size:18px"> SVM </span>

In [35]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = svm.SVC(kernel='linear', C=1, random_state=42)
    clf.fit(X_train_cbow[i],y_train_gsim_L[i])
    y_pred = clf.predict(X_test_cbow[i])
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

[[50  3  1]
 [12 82  2]
 [ 0  0 37]]
Accuracy:   0.895888674565996
Recall:    [0.9184314  0.85017245 0.97756673]
Precision: [0.79531454 0.95822458 0.91560824]
F1_Score:  [0.85212476 0.90076655 0.94532556]


In [36]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = KNeighborsClassifier(n_neighbors = 20, metric='cosine')
    clf.fit(X_train_cbow[i],y_train_gsim_L[i])
    y_pred = clf.predict(X_test_cbow[i])
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to a

[[52  6  2]
 [10 79  1]
 [ 0  0 37]]
Accuracy:   0.8885147423532654
Recall:    [0.86118768 0.86999667 0.97755153]
Precision: [0.82055445 0.92203188 0.91535808]
F1_Score:  [0.83974043 0.89491033 0.94488966]


# Doc2Vec

In [37]:
# TRAIN PARAMS
# i: 0->4
# train_folds[i],train_folds[i]['category']

X_train_gsim_L = []
y_train_gsim_L = []
for i in range(5):
    train_gsim = [utils.simple_preprocess(x) for x in train_folds[i]['text']]
    train_data_labels = [(x,y) for (x,y) in zip(train_gsim, train_folds[i]['category']) if len(x) > 0]
    X_train_gsim = [x for (x,y) in train_data_labels]
    y_train_gsim = [y for (x,y) in train_data_labels]
    X_train_gsim_L.append(X_train_gsim)
    y_train_gsim_L.append(y_train_gsim)

# TEST PARAMS
# i: 0->4
# test_folds[i],test_folds[i]['category']

X_test_gsim_L = []
y_test_gsim_L = []
for i in range(5):
    test_gsim = [utils.simple_preprocess(x) for x in test_folds[i]['text']]
    test_data_labels = [(x,y) for (x,y) in zip(test_gsim, test_folds[i]['category']) if len(x) > 0]
    X_test_gsim = [x for (x,y) in test_data_labels]
    y_test_gsim = [y for (x,y) in test_data_labels]
    X_test_gsim_L.append(X_test_gsim)
    y_test_gsim_L.append(y_test_gsim)

In [38]:
embedding_size = 50
d2v_Train = []
d2v_Test = []
# sg=0: cbow, sg=1: skipgram
for j in range(5):
    print("Fold")
    train_corpus = [gensim.models.doc2vec.TaggedDocument(X_train_gsim_L[j][i], [i]) for i in range(len(X_train_gsim_L[j]))]
    d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
    d2v_model.build_vocab(train_corpus)
    d2v_model.train(train_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)
    print(len(X_train_gsim_L[j]))
    X_train_d2v = [d2v_model.infer_vector(x) for x in X_train_gsim_L[j]]
    # follwoing line fixed!
    X_test_d2v = [d2v_model.infer_vector(x) for x in X_test_gsim_L[j]]
    d2v_Train.append(X_train_d2v)
    d2v_Test.append(X_test_d2v)

Fold
760
Fold
761
Fold
761
Fold
761
Fold
761


In [39]:
X_train_cbow = []
for i in range(5):
    X_train_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y]).mean(axis = 0) for y in X_train_gsim_L[i]]
    X_train_cbow.append(X_train_cbow_temp)

In [ ]:
X_test_cbow = []
for i in range(5):
    X_test_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y if x in cbow_list[i].wv]).mean(axis = 0) for y in X_test_gsim_L[i]]
    X_test_cbow.append(X_test_cbow_temp)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-c0df0c646ac2>", line 3, in <module>
    X_test_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y if x in cbow_list[i].wv]).mean(axis = 0) for y in X_test_gsim_L[i]]
  File "<ipython-input-40-c0df0c646ac2>", line 3, in <listcomp>
    X_test_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y if x in cbow_list[i].wv]).mean(axis = 0) for y in X_test_gsim_L[i]]
  File "<ipython-input-40-c0df0c646ac2>", line 3, in <listcomp>
    X_test_cbow_temp = [np.array([cbow_list[i].wv[x] for x in y if x in cbow_list[i].wv]).mean(axis = 0) for y in X_test_gsim_L[i]]
  File "D:\anaconda3\lib\site-packages\gensim\models\keyedvectors.py", line 649, in __contains__
    return self.has_index_for(key)
  File "D:\anaconda3\lib\site-packages\gensim\models\keyedvectors.py", line 646, in has_index_fo

<span style="color:black; font-weight:700;font-size:18px"> Logistic </span>

In [ ]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = LogisticRegression(random_state=16,solver='lbfgs', max_iter=1000)
    clf.fit(d2v_Train[i],y_train_gsim_L[i])
    y_pred = clf.predict(d2v_Test[i])
    #print(len(y_pred),len(y_test_gsim_L[i]),len(d2v_Test[i]))
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

<span style="color:black; font-weight:700;font-size:18px"> SVM </span>

In [ ]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = svm.SVC(kernel='linear', C=1, random_state=42)
    clf.fit(X_train_cbow[i],y_train_gsim_L[i])
    y_pred = clf.predict(X_test_cbow[i])
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)

In [ ]:
conf = np.array([[0,0,0],[0,0,0],[0,0,0]])
acc = []
rec = []
prec = []
f1 = []
for i in range(5):
    clf = KNeighborsClassifier(n_neighbors = 20, metric='cosine')
    clf.fit(X_train_cbow[i],y_train_gsim_L[i])
    y_pred = clf.predict(X_test_cbow[i])
    acc.append(metrics.accuracy_score(y_pred, y_test_gsim_L[i]))
    prec.append(metrics.precision_score(y_pred, y_test_gsim_L[i], average=None))
    rec.append(metrics.recall_score(y_pred, y_test_gsim_L[i], average=None))
    f1.append(metrics.f1_score(y_pred, y_test_gsim_L[i], average=None))
    conf_mat = confusion_matrix(y_pred, y_test_gsim_L[i])
    conf = conf + conf_mat
conf_mean = conf/5
conf_mean = conf_mean.astype(int)
rec = np.array(rec)
rec = rec.mean(axis=0)

prec = np.array(prec)
prec = prec.mean(axis=0)

f1 = np.array(f1)
f1 = f1.mean(axis=0)

acc = np.array(acc)
print(conf_mean)
print("Accuracy:  ",acc.mean())
print("Recall:   ",rec)
print("Precision:",prec)
print("F1_Score: ",f1)